In [1]:
import scipy
import scipy.io as sio
import numpy as np
from PIL import Image
import math
import os
from keras.models import Sequential
from keras.layers import *
from keras.layers.advanced_activations import PReLU
#utilities help us transform our data
from keras.utils import * 
from sklearn.cross_validation import train_test_split
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import tensorflow as tf

import cv2
from keras.models import load_model
from statistics import mode
#from utils import preprocess_input
from random import shuffle

Using TensorFlow backend.
D:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
# Load trained CNN model for gender classification

classification_model_path = './crop_img/test_CNN_model.hdf5'
detection_model_path = './crop_img/haarcascade_frontalface_default.xml'
frame_window = 10
gender_labels = {0:'Female',1:'Male'}

face_detection = cv2.CascadeClassifier(detection_model_path)
# load trained model
gender_classifier = load_model(classification_model_path)
gender_window = []
# capture video frames
video_capture = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.startWindowThread()
cv2.namedWindow('window_frame')
while True:
    _, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        # draw rectangle on faces
        cv2.rectangle(gray,(x - (int)(0.2*w), y - (int)(0.3*h)),(x + (int)(1.2*w), y + (int)(1.2*h)),(255,0,0),2)
        # crop faces
        face = gray[y - (int)(0.3*h) : y + (int)(1.2*h), x - (int)(0.2*w) : x + (int)(1.2*w)]
        try:
            face = cv2.resize(face, (64, 64))
        except:
            continue
        face = np.expand_dims(face,0)
        face = np.expand_dims(face,-1)
        gender_arg = np.argmax(gender_classifier.predict(face))
        gender = gender_labels[gender_arg]
        gender_window.append(gender)

        if len(gender_window) >= frame_window:
            gender_window.pop(0)
        try:
            gender_mode = mode(gender_window)
        except:
            continue
        # put text label on face
        cv2.putText(gray,gender_mode,(x,y-30), font, .7,(255,0,0),1,cv2.LINE_AA)
    try:
        cv2.imshow('window_frame', gray)
    except:
        continue

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

D:\Anaconda\lib\site-packages\keras\engine\topology.py:1269: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(name="convolution2d_1", activity_regularizer=None, trainable=True, input_dtype="float32", batch_input_shape=[None, 64,..., activation="linear", kernel_size=(7, 7), filters=16, strides=[1, 1], padding="same", data_format="channels_last", kernel_initializer="glorot_uniform", kernel_regularizer=None, bias_regularizer=None, kernel_constraint=None, bias_constraint=None, use_bias=True)`
  return cls(**config)
D:\Anaconda\lib\site-packages\keras\engine\topology.py:1269: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(trainable=True, name="prelu_1", alpha_initializer="zero")`
  return cls(**config)
D:\Anaconda\lib\site-packages\keras\engine\topology.py:1269: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(gamma_regularizer=None, name="batchnormalization_1", epsilon=0.001, trainable=True, beta_regularizer=None